In [1]:
# import sys
# !{sys.executable} -m pip install flask
# !{sys.executable} -m pip install flask-restful
from flask import Flask
from flask_restful import Api, Resource, reqparse

In [2]:
import pandas as pd
import datetime

In [3]:
bids = []
asks = []
fills = []
users = {'Michael': 0, 'Test': 0} #cash per user

In [4]:
def order_flow(bids, asks):
    global fills
    temp = []

    for bid in bids:
        bid_id = bid['user_id']
        bid_volume = bid['volume']
        bid_price = bid['price']
        bid_time = bid['time']
        for ask in asks:
            ask_id = ask['user_id']
            ask_volume = ask['volume']
            ask_price = ask['price']
            ask_time = ask['time']

            if bid_id != ask_id and bid['security_id'] == ask['security_id']:
                if ask_price <= bid_price:
                    fills.append(create_filled(bid,ask))
                    sub = min(bid['volume'],ask['volume'])
                    bid['volume'] -= sub
                    ask['volume'] -= sub

                    bids = list(filter(lambda i: i['volume'] != 0, bids)) 
                    asks = list(filter(lambda i: i['volume'] != 0, asks)) 

    bids = list(filter(lambda i: i['volume'] != 0, bids)) 
    asks = list(filter(lambda i: i['volume'] != 0, asks)) 

    temp.append(bids)
    temp.append(asks)

    return temp

In [5]:
def create_bid_ask(security,user_id, volume, price):
    temp = {}
    temp['security_id'] = security
    temp['user_id'] = user_id
    temp['volume'] = volume
    temp['price'] = price
    temp['time'] = datetime.datetime.now()
    return temp

In [6]:
def create_filled(bid, ask):
    global users
    price = 0
    if bid['time'] < ask['time']:
        price = min(bid['price'],ask['price'])
    else:
        price = max(ask['price'],bid['price'])

    temp = {}
    temp['bid_id'] = bid['user_id']
    temp['ask_id'] = ask['user_id']
    temp['volume'] = min(bid['volume'],ask['volume'])
    temp['price'] = price
    temp['time'] = datetime.datetime.now()
    temp['bid_time'] = bid['time']
    temp['ask_time'] = ask['time']
    temp['security_id'] = bid['security_id']

    users[bid['user_id']] -= price * min(bid['volume'],ask['volume'])
    users[ask['user_id']] += price * min(bid['volume'],ask['volume'])

    return temp

# Example Order

In [7]:
bids.append(create_bid_ask('test_security1','Michael',10,1)) #add a bid, 10 lot to buy at $1

In [8]:
bids

[{'security_id': 'test_security1',
  'user_id': 'Michael',
  'volume': 10,
  'price': 1,
  'time': datetime.datetime(2019, 11, 16, 16, 55, 39, 73128)}]

We can add another order with a different security id

In [9]:
bids.append(create_bid_ask('test_security2','Michael',100,5)) #add a bid, 100 lot to buy at $5

In [10]:
bids

[{'security_id': 'test_security1',
  'user_id': 'Michael',
  'volume': 10,
  'price': 1,
  'time': datetime.datetime(2019, 11, 16, 16, 55, 39, 73128)},
 {'security_id': 'test_security2',
  'user_id': 'Michael',
  'volume': 100,
  'price': 5,
  'time': datetime.datetime(2019, 11, 16, 16, 55, 39, 113046)}]

Can make a dataframe

In [11]:
bids_df = pd.DataFrame(bids)

In [12]:
bids_df

,security_id,user_id,volume,price,time
0,test_security1,Michael,10,1,2019-11-16 16:55:39.073128
1,test_security2,Michael,100,5,2019-11-16 16:55:39.113046


Select where security_id = whatever you want, or volume is > x

In [13]:
bids_df.loc[bids_df.security_id == 'test_security1']

,security_id,user_id,volume,price,time
0,test_security1,Michael,10,1,2019-11-16 16:55:39.073128


In [14]:
bids_df.loc[bids_df.volume > 50]

,security_id,user_id,volume,price,time
1,test_security2,Michael,100,5,2019-11-16 16:55:39.113046


Iterate through each row

In [15]:
for index, row in bids_df.iterrows():
    print('Price: ' + str(row['price']) + ' Volume: ' + str(row['volume']) + ' Security: ' + row['security_id'])

Price: 1 Volume: 10 Security: test_security1
Price: 5 Volume: 100 Security: test_security2


# Adding Opposite Order

In [16]:
asks.append(create_bid_ask('test_security1','Test',5,1)) #add an ask, 5 lot to sell at $1

In [17]:
asks

[{'security_id': 'test_security1',
  'user_id': 'Test',
  'volume': 5,
  'price': 1,
  'time': datetime.datetime(2019, 11, 16, 16, 55, 39, 205722)}]

Ideally, the matching engine is run after each order

In [18]:
bids, asks = order_flow(bids, asks)

In [19]:
bids

[{'security_id': 'test_security1',
  'user_id': 'Michael',
  'volume': 5,
  'price': 1,
  'time': datetime.datetime(2019, 11, 16, 16, 55, 39, 73128)},
 {'security_id': 'test_security2',
  'user_id': 'Michael',
  'volume': 100,
  'price': 5,
  'time': datetime.datetime(2019, 11, 16, 16, 55, 39, 113046)}]

In [20]:
asks

[]

In [21]:
fills

[{'bid_id': 'Michael',
  'ask_id': 'Test',
  'volume': 5,
  'price': 1,
  'time': datetime.datetime(2019, 11, 16, 16, 55, 39, 224978),
  'bid_time': datetime.datetime(2019, 11, 16, 16, 55, 39, 73128),
  'ask_time': datetime.datetime(2019, 11, 16, 16, 55, 39, 205722),
  'security_id': 'test_security1'}]

In [22]:
users

{'Michael': -5, 'Test': 5}

# Example DataFrame Practice

In [23]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')

In [24]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [25]:
iris.loc[iris.sepal_length > 7]

,sepal_length,sepal_width,petal_length,petal_width,species
102,7.1,3.0,5.9,2.1,virginica
105,7.6,3.0,6.6,2.1,virginica
107,7.3,2.9,6.3,1.8,virginica
109,7.2,3.6,6.1,2.5,virginica
117,7.7,3.8,6.7,2.2,virginica
118,7.7,2.6,6.9,2.3,virginica
122,7.7,2.8,6.7,2.0,virginica
125,7.2,3.2,6.0,1.8,virginica
129,7.2,3.0,5.8,1.6,virginica
130,7.4,2.8,6.1,1.9,virginica


In [26]:
iris.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

In [27]:
iris.species.unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [28]:
for name, group in iris.groupby('species'):
    print(name + ' average sepal length: ' + str(group.sepal_length.mean()))

setosa average sepal length: 5.005999999999999
versicolor average sepal length: 5.936
virginica average sepal length: 6.587999999999998


In [29]:
setosa = iris.loc[iris.species == 'setosa']

In [30]:
setosa.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [31]:
setosa.sepal_length

0     5.1
1     4.9
2     4.7
3     4.6
4     5.0
5     5.4
6     4.6
7     5.0
8     4.4
9     4.9
10    5.4
11    4.8
12    4.8
13    4.3
14    5.8
15    5.7
16    5.4
17    5.1
18    5.7
19    5.1
20    5.4
21    5.1
22    4.6
23    5.1
24    4.8
25    5.0
26    5.0
27    5.2
28    5.2
29    4.7
30    4.8
31    5.4
32    5.2
33    5.5
34    4.9
35    5.0
36    5.5
37    4.9
38    4.4
39    5.1
40    5.0
41    4.5
42    4.4
43    5.0
44    5.1
45    4.8
46    5.1
47    4.6
48    5.3
49    5.0
Name: sepal_length, dtype: float64

In [32]:
setosa.sepal_length.mean()

5.005999999999999